In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## Split into training and test in advance

In [2]:
import numpy as np
data = np.load("./data/apnea_data.npz")

In [3]:
samples, labels = data['samples'], data['labels']

In [6]:
idx = np.arange(len(labels))

In [7]:
np.random.shuffle(idx)

In [8]:
test_len = int(0.1 * len(idx))

In [9]:
test_len

1701

In [10]:
test_idx = idx[:test_len]
train_idx = idx[test_len:]

In [11]:
len(test_idx)

1701

In [12]:
len(train_idx)

15309

In [13]:
samples.shape

(6000, 17010)

In [14]:
train_samples = samples[:, train_idx]

In [15]:
train_samples.shape

(6000, 15309)

In [16]:
train_labels = labels[train_idx]

In [18]:
test_samples = samples[:, test_idx]

In [19]:
test_labels = labels[test_idx]

In [20]:
path = "./data"
import os

In [21]:
with open(os.path.join(path, "train_apnea_data.npz"), "wb") as f:
    np.savez(f, samples=train_samples, labels=train_labels)

In [22]:
with open(os.path.join(path, "test_apnea_data.npz"), "wb") as f:
    np.savez(f, samples=test_samples, labels=test_labels)

## Load data and train a feedforward model

In [2]:
from lib.mlggm.datamodules import ApneaDataModule

ModuleNotFoundError: No module named 'torch'

In [2]:
dm = ApneaDataModule(
    batch_size=1024, num_workers=4, shuffle=False, path="./data"
)

Size of training set:  15309
Size of validation set:  1701


In [3]:
from lib.mlggm.ff_model import FF

In [4]:
model = FF()

In [5]:
import pytorch_lightning as pl

In [6]:
from lib.mlggm.callback import MetricTracker
cb = MetricTracker()

In [7]:
trainer = pl.Trainer(max_epochs=10, check_val_every_n_epoch=2, callbacks=[cb])

/home/agarmanv/Documents/cnn_graph/cnn_venv/lib/python3.8/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
trainer.fit(model=model, datamodule=dm)


  | Name          | Type              | Params
----------------------------------------------------
0 | loss_fn       | BCEWithLogitsLoss | 0     
1 | hidden_layer  | Linear            | 6.1 M 
2 | nonlin        | Sigmoid           | 0     
3 | readout_layer | Linear            | 1.0 K 
----------------------------------------------------
6.1 M     Trainable params
0         Non-trainable params
6.1 M     Total params
24.584    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/agarmanv/Documents/cnn_graph/cnn_venv/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1933: PossibleUserWarning: The number of training batches (15) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [11]:
cb.elogs_collection

[{'test/loss_step': tensor(0.8174),
  'test/acc_step': tensor(0.3456),
  'test/loss_epoch': tensor(0.8036),
  'test/acc_epoch': tensor(0.3704)},
 {'train/loss_step': tensor(0.6746),
  'train/loss_epoch': tensor(0.6753),
  'test/loss_step': tensor(0.6487),
  'test/acc_step': tensor(0.6544),
  'test/loss_epoch': tensor(0.6691),
  'test/acc_epoch': tensor(0.6296)},
 {'train/loss_step': tensor(0.6746),
  'train/loss_epoch': tensor(0.6753),
  'test/loss_step': tensor(0.6487),
  'test/acc_step': tensor(0.6544),
  'test/loss_epoch': tensor(0.6691),
  'test/acc_epoch': tensor(0.6296)},
 {'train/loss_step': tensor(0.6746),
  'train/loss_epoch': tensor(0.6753),
  'test/loss_step': tensor(0.6487),
  'test/acc_step': tensor(0.6544),
  'test/loss_epoch': tensor(0.6691),
  'test/acc_epoch': tensor(0.6296)},
 {'train/loss_step': tensor(0.6746),
  'train/loss_epoch': tensor(0.6753),
  'test/loss_step': tensor(0.6487),
  'test/acc_step': tensor(0.6544),
  'test/loss_epoch': tensor(0.6691),
  'test/acc_

To see results below with tensorboard, go to your terminal and run:
`tensorboard --logdir=lightning_logs/`<br>
Then run the cell below to see training loss.

In [10]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

Reusing TensorBoard on port 6006 (pid 149707), started 0:01:51 ago. (Use '!kill 149707' to kill it.)